In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonIn2\df_NonIn2FNA24_allstd.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 14463, 1: 10302
Yy_FNA = fnaDEFSDf["type"]  # 0.8562; 1.2020
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.8562)
    elif w == 1:
        sampleFNAW.append(1.2020)


In [4]:
fnaDEFSDf

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246,type
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.614860,0.093588,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.965764,0.757883,0.0,-0.551514,0.0,0.875937,0
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.004224,0.349007,0.404083,0.0,0.0,0.0,0.009835,0.0,0.0,0.0,-0.209668,0.586674,0.088131,0.0,0.008937,0.0,-0.124063,0
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,0.039617,0.575782,0.273707,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.429720,-0.242117,0.0,0.089841,0.0,-0.124063,0
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,0.021044,0.611795,0.120438,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.470107,0.249002,0.0,0.076477,0.0,-0.124063,0
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.512301,-0.203629,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.456817,-0.242117,0.0,-0.115004,0.0,-0.124063,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24760,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,0.0,0.322806,-0.222611,0.242797,0.0,0.0,0.0,0.380246,0.0,0.0,0.0,0.372991,-0.034236,-0.242117,0.0,0.072633,0.0,-0.124063,1
24761,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,0.0,0.254461,0.146804,0.218574,0.0,0.0,0.0,0.325864,0.0,0.0,0.0,0.291273,-0.034236,-0.242117,0.0,0.076999,0.0,-0.124063,1
24762,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,0.0,0.330261,0.212512,0.267351,0.0,0.0,0.0,0.256718,0.0,0.0,0.0,0.359895,-0.034236,-0.242117,0.0,0.117832,0.0,-0.124063,1
24763,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,0.0,0.354435,-0.222611,0.276726,0.0,0.0,0.0,0.380465,0.0,0.0,0.0,0.328792,-0.034236,-0.242117,0.0,0.111668,0.0,-0.124063,1


In [5]:
## define functions for performace evaluation ##
    # Maximum absolute error
# mean square error (MSE) calculation
# Root mean square error (RMSE) calculation
def errorDetermination(arrRi, predictedRi):
    sumAE = 0
    maxAE = 0
    for i in range(len(predictedRi)):
        AE = abs(arrRi[i] - predictedRi[i])
        if (AE > maxAE):
            maxAE = AE
        sumAE += (AE ** 2)
        
    MSE = sumAE / len(predictedRi)
    RMSE = MSE ** 0.5
    return maxAE, MSE, RMSE

#
# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [6]:
fnaDEFSDf[fnaDEFSDf.columns[1:-1]]

,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246
0,0.0,-0.382212,0.614860,0.093588,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.965764,0.757883,0.0,-0.551514,0.0,0.875937
1,0.0,-0.004224,0.349007,0.404083,0.0,0.0,0.0,0.009835,0.0,0.0,0.0,-0.209668,0.586674,0.088131,0.0,0.008937,0.0,-0.124063
2,0.0,0.039617,0.575782,0.273707,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.429720,-0.242117,0.0,0.089841,0.0,-0.124063
3,0.0,0.021044,0.611795,0.120438,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.470107,0.249002,0.0,0.076477,0.0,-0.124063
4,0.0,-0.382212,0.512301,-0.203629,0.0,0.0,0.0,-0.305501,0.0,0.0,0.0,-0.209668,0.456817,-0.242117,0.0,-0.115004,0.0,-0.124063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24760,0.0,0.322806,-0.222611,0.242797,0.0,0.0,0.0,0.380246,0.0,0.0,0.0,0.372991,-0.034236,-0.242117,0.0,0.072633,0.0,-0.124063
24761,0.0,0.254461,0.146804,0.218574,0.0,0.0,0.0,0.325864,0.0,0.0,0.0,0.291273,-0.034236,-0.242117,0.0,0.076999,0.0,-0.124063
24762,0.0,0.330261,0.212512,0.267351,0.0,0.0,0.0,0.256718,0.0,0.0,0.0,0.359895,-0.034236,-0.242117,0.0,0.117832,0.0,-0.124063
24763,0.0,0.354435,-0.222611,0.276726,0.0,0.0,0.0,0.380465,0.0,0.0,0.0,0.328792,-0.034236,-0.242117,0.0,0.111668,0.0,-0.124063


In [7]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [8]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [9]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

## 2.2. PCA Analysis

In [12]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_FNA):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_FNA.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonIn2/PCA_NonInFNA2_all.tif", bbox_inches = 'tight')

    """components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()"""

In [13]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [14]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [15]:
y_FNA = df_ROI_for_ML_Opti_FNA["type"]

## 3.2 Prediction by Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [16]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [17]:
import joblib as jl

## 4.1 Prediction

In [18]:
## deploy models for FNA set ##
model_xgb = jl.load(r"H:\3_output_raMSIn\\modelXGBoost.joblib")
#
## deploy model ##
predicted0n1_FNA = model_xgb.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonIn2\df_NonIn2FNA24_allstd_0n1.csv"
fnaDEFSDf.to_csv(savePath)

## 4.2 p(0) p(1)

In [19]:
pTP_FNA = model_xgb.predict_proba(X_FNA)
fnaDEFSDf["p(0)"] = list(pd.DataFrame(pTP_FNA)[0])
fnaDEFSDf["p(1)"] = list(pd.DataFrame(pTP_FNA)[1])
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonIn2\df_NonIn2FNA24_allstd_0n1_pTP.csv"
fnaDEFSDf.to_csv(savePath)